In [4]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import imblearn
print(imblearn.__version__)


0.14.0


In [6]:
pip install matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.1 MB 1.9 MB/s eta 0:00:04
   ------ --------------------------------- 1.3/8.1 MB 2.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.1 MB 2.2 MB/s eta 0:00:03
   ---------- ----------------------------- 2.1/8.1 MB 2.3 MB/s eta 0:00:03
   ------------ --------------------------- 2.6/8.1 MB 2.3 MB/s eta 0:00:03
   --------------- ------------------------ 3.1/8.1 MB 2.4 MB/s eta 0:00:03
   ------------------- -------------------- 3.9/8.1 MB 2.4 MB/s eta 0:00:02
   --------------------- ------------------ 4.5/8.1 MB 2.4 MB/s eta 0:00:02
   ------------------------ --------------- 5.0/8.1 MB 2.5 MB/s eta 0:00:02
   --------------------------- ------------ 5.5/8.1 MB 2.5 MB/s eta 0:00:02
   ----------------------------- ---------- 6.0/8.1 MB 2.5 MB/s eta 0:00:01
   -----------------------

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

In [8]:
import matplotlib
print(matplotlib.__version__)

3.10.7


In [9]:
# Load dataset
df = pd.read_csv("cleaned_data.csv")

# Display first few rows
print(df.head())

          utc_timestamp        cet_cest_timestamp  \
0  2014-12-31T23:00:00Z  2015-01-01T00:00:00+0100   
1  2015-01-01T00:00:00Z  2015-01-01T01:00:00+0100   
2  2015-01-01T01:00:00Z  2015-01-01T02:00:00+0100   
3  2015-01-01T02:00:00Z  2015-01-01T03:00:00+0100   
4  2015-01-01T03:00:00Z  2015-01-01T04:00:00+0100   

   es_load_actual_entsoe_transparency  es_load_forecast_entsoe_transparency  \
0                            0.482170                              0.470935   
1                            0.482170                              0.470935   
2                            0.233405                              0.256014   
3                            0.172684                              0.219679   
4                            0.129828                              0.184009   

   es_solar_generation_actual  es_wind_onshore_generation_actual  \
0                    0.222208                           0.362431   
1                    0.222208                           0.362431   
2 

In [10]:
# Drop missing values (if any)
df = df.dropna()

# Remove duplicates
df = df.drop_duplicates()


In [11]:
print(df.columns)

Index(['utc_timestamp', 'cet_cest_timestamp',
       'es_load_actual_entsoe_transparency',
       'es_load_forecast_entsoe_transparency', 'es_solar_generation_actual',
       'es_wind_onshore_generation_actual', 'es_temperature',
       'es_radiation_direct_horizontal', 'es_radiation_diffuse_horizontal'],
      dtype='object')


In [12]:
print(df.columns.tolist())

['utc_timestamp', 'cet_cest_timestamp', 'es_load_actual_entsoe_transparency', 'es_load_forecast_entsoe_transparency', 'es_solar_generation_actual', 'es_wind_onshore_generation_actual', 'es_temperature', 'es_radiation_direct_horizontal', 'es_radiation_diffuse_horizontal']


In [19]:
data = pd.read_csv("cleaned_data.csv")
data.head()
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42775 entries, 0 to 42774
Data columns (total 9 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   utc_timestamp                         42775 non-null  object 
 1   cet_cest_timestamp                    42775 non-null  object 
 2   es_load_actual_entsoe_transparency    42775 non-null  float64
 3   es_load_forecast_entsoe_transparency  42775 non-null  float64
 4   es_solar_generation_actual            42775 non-null  float64
 5   es_wind_onshore_generation_actual     42775 non-null  float64
 6   es_temperature                        42775 non-null  float64
 7   es_radiation_direct_horizontal        42775 non-null  float64
 8   es_radiation_diffuse_horizontal       42775 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.9+ MB


In [20]:
data = data.interpolate(method='linear')  # fills missing values smoothly


C:\Users\Admin\AppData\Local\Temp\ipykernel_7012\2805199938.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')  # fills missing values smoothly


In [21]:
data.drop(columns=['utc_timestamp'], inplace=True)


In [23]:
data['cet_cest_timestamp'] = pd.to_datetime(
    data['cet_cest_timestamp'], 
    errors='coerce',          # handles any malformed dates
    utc=True                  # keeps timezone consistent
)


In [24]:
data['cet_cest_timestamp'] = data['cet_cest_timestamp'].dt.tz_convert('Europe/Madrid')

In [25]:
data['hour'] = data['cet_cest_timestamp'].dt.hour
data['day'] = data['cet_cest_timestamp'].dt.day
data['month'] = data['cet_cest_timestamp'].dt.month
data['year'] = data['cet_cest_timestamp'].dt.year
data['day_of_week'] = data['cet_cest_timestamp'].dt.dayofweek


In [27]:
data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x>=5 else 0)


In [30]:
data['prev_hour_load'] = data['es_load_actual_entsoe_transparency'].shift(1)
data['prev_day_load'] = data['es_load_actual_entsoe_transparency'].shift(24)
data['rolling_mean_6h'] = data['es_load_actual_entsoe_transparency'].rolling(window=6).mean()
data['rolling_mean_24h'] = data['es_load_actual_entsoe_transparency'].rolling(window=24).mean()


In [ ]:
data.dropna(inplace=True)